In [ ]:
from google.colab import files
import io
import pandas as pd
import numpy as np

# Load the file into a DataFrame
df = pd.read_csv('healthcare_messy_data.csv')


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", use_auth_token="ENTER_YOUR_TOKEN")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", use_auth_token="ENTER_YOUR_TOKEN")

In [ ]:
# Model identifier on Hugging Face
model_id = "meta-llama/Llama-3.2-1B"  # Make sure you accept the terms of use

# Loading tokenizer and template
tokenizer = AutoTokenizer.from_pretrained(model_id, token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map="auto"
)

# Creation of the generation pipeline
llama_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

print("✅ LLaMA model successfully loaded!")

In [ ]:
# Generate a text prompt to send to the model
# Extract key statistics
rows = len(df)
columns = len(df.columns)
missing_fixed = df.isnull().sum().sum()
summary_stats = df.describe().to_dict()  # converts numerical stats into a dictionary

prompt = f"""
Below is a summary of key statistics from the dataset:
- Total rows: {rows}
- Total columns: {columns}
- Number of missing values fixed: {missing_fixed}

Please give three key insights in bullet form, under 100 words.
"""



In [ ]:
# Text generation with LLaMA
output = llama_pipeline(prompt, max_new_tokens=150, temperature=0.7)

# Display generated summary
print("Summary generated by LLaMA :\n", output[0]['generated_text'])

In [ ]:
# Summary stats
summary_stats = df.describe().transpose()

# Convert stats to string
stats_text = summary_stats.to_string()
prompt = f"""
You are a data analyst. Analyze the following dataset summary and provide a short report on key trends, outliers, or correlations.

- Total rows: {rows}
- Total columns: {columns}
- Number of missing values fixed: {missing_fixed}

Statistical Summary:
{stats_text}

Write your analysis in a concise and professional tone.
"""

output = llama_pipeline(prompt, max_new_tokens=300, temperature=0.7)

# Display the result
print("Generated summary by LLaMA:\n", output[0]['generated_text'])